<div align = "center">

# **Exploración de Base Final Anual**

</div>

## Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Carga de Datos

In [ ]:
# Cargar base final
df = pd.read_csv('../data/tmp/base_final_anual.csv')

print(f"Dimensiones: {df.shape}")
print(f"\nColumnas:")
print(df.dtypes)
df.head()

## Exploración General

In [ ]:
print("=" * 80)
print("RESUMEN ESTADÍSTICO GENERAL")
print("=" * 80)

print(f"\nTotal de registros: {len(df):,}")
print(f"Pólizas únicas: {df['POLIZA'].nunique():,}")
print(f"Amparos únicos: {df['AMPARO'].nunique()}")

print(f"\nAmparos disponibles:")
print(df['AMPARO'].value_counts())

## Análisis por Amparo (Cobertura)

In [ ]:
print("=" * 80)
print("ESTADÍSTICAS POR AMPARO")
print("=" * 80)

# Métricas por amparo
stats_amparo = df.groupby('AMPARO').agg({
    'POLIZA': 'count',  # Número de períodos
    'TUVO_SINIESTRO': ['sum', 'mean'],  # Siniestros y frecuencia
    'EXPOSICION_ANUAL': 'sum',  # Exposición total
    'SEVERIDAD_ANUAL': ['sum', 'mean', 'std']  # Severidad
}).round(4)

stats_amparo.columns = ['N_PERIODOS', 'N_SINIESTROS', 'FRECUENCIA', 
                        'EXPOSICION_TOTAL', 'SEVERIDAD_TOTAL', 
                        'SEVERIDAD_MEDIA', 'SEVERIDAD_STD']

# Calcular prima pura = severidad_total / exposicion_total
stats_amparo['PRIMA_PURA'] = stats_amparo['SEVERIDAD_TOTAL'] / stats_amparo['EXPOSICION_TOTAL']

stats_amparo = stats_amparo.sort_values('N_PERIODOS', ascending=False)
stats_amparo

In [ ]:
# Visualización de frecuencia por amparo
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Frecuencia
stats_amparo['FRECUENCIA'].sort_values().plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('Frecuencia de Siniestros por Amparo')
axes[0].set_xlabel('Frecuencia (siniestros/período)')

# Exposición
stats_amparo['EXPOSICION_TOTAL'].sort_values().plot(kind='barh', ax=axes[1], color='darkgreen')
axes[1].set_title('Exposición Total por Amparo')
axes[1].set_xlabel('Exposición ($)')

plt.tight_layout()
plt.show()

## Clustering de Coberturas

In [ ]:
print("=" * 80)
print("CLUSTERING DE COBERTURAS POR COMPORTAMIENTO")
print("=" * 80)

# Preparar features para clustering
features_cluster = stats_amparo[['FRECUENCIA', 'SEVERIDAD_MEDIA', 'PRIMA_PURA', 'N_PERIODOS']].copy()
features_cluster = features_cluster.fillna(0)

# Normalizar
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_cluster)

print(f"\nFeatures para clustering:")
print(features_cluster)

In [ ]:
# Determinar número óptimo de clusters (método del codo)
if len(features_cluster) >= 3:
    inertias = []
    silhouettes = []
    K_range = range(2, min(len(features_cluster), 6))
    
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(features_scaled)
        inertias.append(kmeans.inertia_)
        silhouettes.append(silhouette_score(features_scaled, kmeans.labels_))
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].plot(K_range, inertias, 'bo-')
    axes[0].set_xlabel('Número de Clusters (k)')
    axes[0].set_ylabel('Inercia')
    axes[0].set_title('Método del Codo')
    
    axes[1].plot(K_range, silhouettes, 'go-')
    axes[1].set_xlabel('Número de Clusters (k)')
    axes[1].set_ylabel('Silhouette Score')
    axes[1].set_title('Silhouette Score por k')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nSilhouette scores: {dict(zip(K_range, [round(s, 3) for s in silhouettes]))}")
else:
    print("Muy pocos amparos para clustering")

In [ ]:
# Aplicar clustering con k óptimo
k_optimo = 3  # Ajustar según resultados anteriores

if len(features_cluster) >= k_optimo:
    kmeans = KMeans(n_clusters=k_optimo, random_state=42, n_init=10)
    stats_amparo['CLUSTER'] = kmeans.fit_predict(features_scaled)
    
    print(f"\nAsignación de clusters (k={k_optimo}):")
    print("=" * 60)
    for cluster in range(k_optimo):
        amparos_cluster = stats_amparo[stats_amparo['CLUSTER'] == cluster].index.tolist()
        print(f"\nCluster {cluster}:")
        for amparo in amparos_cluster:
            freq = stats_amparo.loc[amparo, 'FRECUENCIA']
            n = stats_amparo.loc[amparo, 'N_PERIODOS']
            print(f"  - {amparo} (freq={freq:.4f}, n={n:,.0f})")

In [ ]:
# Visualización de clusters
if 'CLUSTER' in stats_amparo.columns:
    fig, ax = plt.subplots(figsize=(10, 8))
    
    scatter = ax.scatter(
        stats_amparo['FRECUENCIA'], 
        stats_amparo['SEVERIDAD_MEDIA'],
        c=stats_amparo['CLUSTER'],
        s=stats_amparo['N_PERIODOS'] / stats_amparo['N_PERIODOS'].max() * 500 + 50,
        cmap='viridis',
        alpha=0.7,
        edgecolors='black'
    )
    
    # Etiquetas
    for amparo in stats_amparo.index:
        ax.annotate(
            amparo[:20],  # Truncar nombres largos
            (stats_amparo.loc[amparo, 'FRECUENCIA'], 
             stats_amparo.loc[amparo, 'SEVERIDAD_MEDIA']),
            fontsize=8,
            ha='center',
            va='bottom'
        )
    
    ax.set_xlabel('Frecuencia')
    ax.set_ylabel('Severidad Media')
    ax.set_title('Clustering de Amparos por Frecuencia y Severidad')
    plt.colorbar(scatter, label='Cluster')
    plt.tight_layout()
    plt.show()

## Resumen de Clusters para Modelación

In [ ]:
if 'CLUSTER' in stats_amparo.columns:
    print("=" * 80)
    print("RESUMEN DE CLUSTERS PARA MODELACIÓN")
    print("=" * 80)
    
    resumen_clusters = stats_amparo.groupby('CLUSTER').agg({
        'N_PERIODOS': 'sum',
        'N_SINIESTROS': 'sum',
        'FRECUENCIA': 'mean',
        'SEVERIDAD_MEDIA': 'mean',
        'EXPOSICION_TOTAL': 'sum'
    }).round(4)
    
    resumen_clusters['N_AMPAROS'] = stats_amparo.groupby('CLUSTER').size()
    
    print("\n")
    print(resumen_clusters)
    
    print("\n" + "=" * 80)
    print("RECOMENDACIÓN")
    print("=" * 80)
    print("\nBasado en el clustering, se recomienda modelar:")
    print("- Clusters con suficientes datos (N_PERIODOS > 1000)")
    print("- Clusters con siniestralidad observable (N_SINIESTROS > 10)")